In [22]:
from sqlalchemy import create_engine
import os
import time
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load API key for data.gov

with open('data_gov_api.txt', 'r') as file:
    api_key = file.read().split('\n')[0] # Remove new line

conn = create_engine('postgresql://ubuntu@52.53.236.232:5432/collegesc')

data_dir = os.path.join(os.path.curdir, "data")
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

## Helper functions and dictionaries

In [214]:
def find_max_string_length(pd_series):
    """
    For string column in pandas Series, find the maximum length of thes string
    """
    return pd_series.map(lambda x: len(str(x))).max()

def call_collegesc_api(category, api_key, conn, per_page = 50, year = 2013):
    """
    category: the category to be downloaded
    api_key: key for the college sc api
    conn: sql_alchemy engine for querying data definitions
    per_page: number of entries per api call (max: 100)
    year: year to be called (default: 2013)
    calls the college scordcard api given a particular category
    and returns a dataframe transformed from json
    """
    sql_query = """SELECT
                    developername
                FROM 
                    datadefinitions 
                WHERE devcategory ='"""
    
    sql_query += category+"';"
    
    fields = [fields[0] for fields in conn.execute(sql_query).fetchall()]

    
    if(category == "school"):
        join_string = "," + category + "."
        fields_string = "id,location.lat,location.lon" + join_string + join_string.join(fields)
    elif(category == "academics"):
        join_string = "," + category + "."
        fields_string = "id" + join_string + join_string.join(fields)
    else:
        join_string = "," + str(year) + "." + category + "."
        fields_string = "id" + join_string + join_string.join(fields)
    
    college_sc_url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
    api_query = {'api_key' : api_key, '_fields' : fields_string, '_per_page' : per_page}
    
    

    req = requests.get(college_sc_url, params=api_query)
    print(req)
    
    pages = req.json()["metadata"]["total"] // per_page
    
    category_df = pd.DataFrame(req.json()["results"])
    
    for page in range(1, pages+1):
        
        api_query["_page"] = page
        i = 0
        while (i < 10):
            try:
                req = requests.get(college_sc_url, params=api_query)
        
                temp_df = pd.DataFrame(req.json()["results"])
        
                category_df = category_df.append(temp_df)
                
                i = 0
                break
            except:
                print("Connection or query error")
                print(req)
                i += 1
                time.sleep(2)
        
        time.sleep(2)
    
    category_df = category_df.reset_index(drop=True)
    
    category_df.columns = category_df.columns.str.replace(category+"\.", "", n = 1)
    category_df.columns = category_df.columns.str.replace(str(year)+"\.", "", n = 1)
        
    
    return category_df
    
def check_highnull_columns(df, threshold=1000):
    """
    takes a dataframe and checks which columns
    have lower than threshold nulls
    returns partial dataframe info with columns with low nulls
    """
    return df.loc[:, df.isnull().sum() < threshold].info()

sql_type_dict = {'integer' : 'integer', 'float' : 'real', 'string' : 'text', 'autocomplete' : 'text'}

def create_datetype_dict(conn, category, type_dict):
    """
    Generates sql query to conn with category to get
    API data categories and convert to Postgres types
    with type dictionary
    Returns type dictionary for that category to generate
    SQL table
    """
    
    datatype_query = """SELECT 
                        developername, datatype
                    FROM
                        datadefinitions
                    WHERE
                        devcategory = '"""
                    
    datatype_query += category+"';"
    
    type_defs = conn.execute(datatype_query).fetchall()
    
    fields = [x[0] for x in type_defs]
    entries = [type_dict[x[1]] for x in type_defs]
    
    sql_type_def_dict = dict(zip(fields, entries))
    sql_type_def_dict['id'] = 'integer'
    
    return sql_type_def_dict
    

## Download College Scorecard Data Dictionary

In [111]:


data_dict_url = "https://collegescorecard.ed.gov/assets/CollegeScorecardDataDictionary.xlsx"

data_dict_file = data_dict_url.split('/')[-1]

data_dict_path = os.path.join(data_dir, data_dict_file)

if not os.path.exists(data_dict_path):
    req = requests.get(data_dict_url)
    
    with open(data_dict_path, 'wb') as file:
        file.write(req.content)


## Load into memory and create table to define variables

In [113]:
sc_data_dict = pd.read_excel(data_dict_path, sheetname='data_dictionary')

In [114]:
sc_data_dict.iloc[300]

NAME OF DATA ELEMENT       Percentage of degrees awarded in Architecture ...
dev-category                                                       academics
developer-friendly name                      program_percentage.architecture
API data type                                                          float
VARIABLE NAME                                                         PCIP04
VALUE                                                                    NaN
LABEL                                      Architecture and Related Services
SOURCE                                                                 IPEDS
NOTES                                        Shown/used on consumer website.
Name: 300, dtype: object

In [115]:
sc_data_dict.iloc[25:28, :]

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,VALUE,LABEL,SOURCE,NOTES
25,Control of institution,school,ownership,integer,CONTROL,1.0,Public,IPEDS,Shown/used on consumer website.
26,NaN,school,NaN,NaN,NaN,2.0,Private nonprofit,NaN,NaN
27,NaN,school,NaN,NaN,NaN,3.0,Private for-profit,NaN,NaN


In [116]:
sc_data_dict_nv = sc_data_dict.drop(['VALUE', 'LABEL'], axis = 1)

In [117]:
sc_data_dict_nv.columns = ['Name', 'DevCategory', 'DeveloperName', 'DataType', 'VarName', 'Source', 'Notes']

In [118]:
sc_data_dict_nv = sc_data_dict_nv[sc_data_dict_nv['Name'].notnull()]

In [119]:
sc_data_dict_nv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1734 entries, 0 to 1974
Data columns (total 7 columns):
Name             1734 non-null object
DevCategory      1734 non-null object
DeveloperName    1734 non-null object
DataType         1734 non-null object
VarName          1734 non-null object
Source           1734 non-null object
Notes            274 non-null object
dtypes: object(7)
memory usage: 108.4+ KB


### Find maximum string length in each column so we can define SQL table

In [120]:
string_length_dict = dict()
for column in sc_data_dict_nv.columns:
    string_length_dict[column] = find_max_string_length(sc_data_dict_nv[column])

In [109]:
sc_data_dict_nv.to_csv('datadef.csv', index=False)

In [121]:

query = """CREATE TABLE IF NOT EXISTS datadefinitions (
"""
for column, length in string_length_dict.items():
    query += column + " varchar("+str(length)+") DEFAULT NULL,\n"
    
query += """
PRIMARY KEY (developername)
);"""

In [122]:
query

'CREATE TABLE IF NOT EXISTS datadefinitions (\nName varchar(229) DEFAULT NULL,\nDevCategory varchar(10) DEFAULT NULL,\nDeveloperName varchar(73) DEFAULT NULL,\nDataType varchar(12) DEFAULT NULL,\nVarName varchar(32) DEFAULT NULL,\nSource varchar(11) DEFAULT NULL,\nNotes varchar(2109) DEFAULT NULL,\n\nPRIMARY KEY (developername)\n);'

In [112]:
conn.execute(query)

In [130]:
query2 = """COPY 
                datadefinitions
            FROM 
                '/home/ubuntu/Notebooks/CollegeClassification/datadef.csv'
            DELIMITER ',' CSV HEADER;"""

In [131]:
conn.execute(query2)

In [14]:
query3 = """SELECT * FROM datadefinitions;"""

In [15]:
data_definitions = pd.read_sql_query(query3, conn)

## Use API to gather data for 2013

In [215]:
datatype_query = """SELECT
                        *
                    FROM 
                        datadefinitions
                    WHERE
                        devcategory='root';"""

In [216]:
conn.execute(datatype_query).fetchall()

[('Unit ID for institution', 'root', 'id', 'integer', 'UNITID', 'IPEDS', 'Shown/used on consumer website.'),
 ('8-digit OPE ID for institution', 'root', 'ope8_id', 'integer', 'OPEID', 'IPEDS', 'Shown/used on consumer website.'),
 ('6-digit OPE ID for institution', 'root', 'ope6_id', 'integer', 'OPEID6', 'IPEDS', 'Shown/used on consumer website.'),
 ('Latitude', 'root', 'location.lat', 'float', 'LATITUDE', 'IPEDS', None),
 ('Longitude', 'root', 'location.lon', 'float', 'LONGITUDE', 'IPEDS', None)]

In [217]:
autocomplete_query = """SELECT
                            *
                        FROM
                            datadefinitions
                        WHERE
                            datatype='autocomplete'"""

In [218]:
conn.execute(autocomplete_query).fetchall()

[('Institution name', 'school', 'name', 'autocomplete', 'INSTNM', 'IPEDS', 'Shown/used on consumer website.'),
 ('City', 'school', 'city', 'autocomplete', 'CITY', 'IPEDS', 'Shown/used on consumer website.'),
 ('Institution name aliases', 'school', 'alias', 'autocomplete', 'ALIAS', 'IPEDS', None)]

In [219]:
category_query = """SELECT DISTINCT
                        devcategory
                    FROM
                        datadefinitions;"""

In [220]:
categories = [x[0] for x in conn.execute(category_query).fetchall()]

In [221]:
categories = categories[1:]

In [222]:
datatype_query = """SELECT DISTINCT
                        datatype
                    FROM
                        datadefinitions;"""

In [223]:
conn.execute(datatype_query).fetchall()

[('integer',), ('string',), ('autocomplete',), ('float',)]

#### Remove categories with mostly missing data

In [224]:
categories.remove('academics')
categories.remove('earnings')
categories.remove('completion')
categories

['repayment', 'admissions', 'aid', 'student', 'cost', 'school']

#### Gather CSV data using API calls

In [225]:
for category in categories:
    if((category == 'academics') | (category == 'earnings') | (category == 'completion')):
        continue
    print("Starting "+category)
    category_file = os.path.join(data_dir, category + ".csv")
    if not os.path.exists(category_file):
        category_df = call_collegesc_api(category, api_key, conn)
        category_df.to_csv(category_file, index=False)
    print("Completed "+category)

Starting repayment
Completed repayment
Starting admissions
Completed admissions
Starting aid
Completed aid
Starting student
<Response [200]>
Completed student
Starting cost
Completed cost
Starting school
Completed school


## Check resulting CSV files before filling out SQL Database

In [227]:
for category in categories:
    cat_file = os.path.join(data_dir, category+'.csv')
    category_df = pd.read_csv(cat_file)
    print(category)
    print(check_highnull_columns(category_df, threshold=1000))
    print()
    

repayment
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 3 columns):
3_yr_repayment.female_students_rate    7488 non-null float64
3_yr_repayment.male_students_rate      7488 non-null float64
id                                     7703 non-null int64
dtypes: float64(2), int64(1)
memory usage: 180.6 KB
None

admissions
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 1 columns):
id    7703 non-null int64
dtypes: int64(1)
memory usage: 60.3 KB
None

aid
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 3 columns):
federal_loan_rate    6806 non-null float64
pell_grant_rate      6806 non-null float64
id                   7703 non-null int64
dtypes: float64(2), int64(1)
memory usage: 180.6 KB
None

student
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 30 columns):
Unnamed: 0                                     

Not worth using admissions or cost

In [68]:
sql_table_cat = list(categories)

In [69]:
sql_table_cat.remove('admissions')
sql_table_cat.remove('cost')

In [70]:
sql_table_cat

['repayment', 'aid', 'student', 'school']

#### Create tables (Do not use this, pandas converted intergers with nan to floats)

In [127]:
# for category in sql_table_cat:
#     prim_key = 'id'
#     if (category == 'school'):
#         prim_key += ",zip"

#     cat_type_dict = create_datetype_dict(conn, category, sql_type_dict)
#     cat_create_query = "CREATE TABLE IF NOT EXISTS "+ category +" (\n "

#     for field, var_type in cat_type_dict.items():
#         cat_create_query += '"'+ field + '" ' + var_type + ' DEFAULT NULL,\n '

#     cat_create_query += "PRIMARY KEY ("+prim_key+")\n);"
#     conn.execute(cat_create_query)

#### Fill tables with CSV

In [258]:
for category in sql_table_cat:
    print(category)
    category_path = os.path.join(data_dir, category+'.csv')
    category_df = pd.read_csv(category_path)
    try:
        category_df.to_sql(category, conn, if_exists='fail')
    except:
        print(category+" was already added.")
    

repayment
aid
student
school


#### Verify table contents

In [273]:
example_select = """SELECT * FROM school LIMIT 10;"""

In [274]:
pd.read_sql_query(example_select, conn)

,index,id,location.lat,location.lon,accreditor,alias,branches,carnegie_basic,carnegie_size_setting,carnegie_undergrad,...,price_calculator_url,region_id,religious_affiliation,url,state,state_fips,tuition_revenue_per_fte,under_investigation,women_only,zip
0,0,121983,37.781238,-122.267523,Western Association of Schools and Colleges Se...,None,28,26.0,6.0,5.0,...,tcc.noellevitz.com/edmc/Argosy University Net ...,8,-2.0,www.argosy.edu/sanfrancisco,CA,6,12380.0,0,0.0,94501
1,1,120838,34.060700,-118.299841,Accrediting Council for Independent Colleges a...,None,1,29.0,6.0,11.0,...,www.psuca.edu/node/100,8,-2.0,www.psuca.edu,CA,6,13784.0,0,0.0,90010
2,2,434973,39.189991,-76.815804,North Central Association of Colleges and Scho...,None,38,29.0,6.0,11.0,...,www.phoenix.edu/tuition_and_financial_options/...,2,-2.0,www.phoenix.edu,MD,24,10131.0,0,0.0,21045
3,3,436012,40.708670,-73.627681,None,None,2,-2.0,-2.0,-2.0,...,www.franklincareer.edu,2,-2.0,www.franklincareer.edu,NY,36,6935.0,0,0.0,11550
4,4,436021,42.986115,-87.870663,Montessori Accreditation Council for Teacher E...,None,1,-2.0,-2.0,-2.0,...,www.montessori6-12ami.org,3,-2.0,www.montessori6-12ami.org,WI,55,6063.0,0,0.0,53207
5,5,436030,38.839695,-76.948468,National Accrediting Commission of Cosmetology...,None,1,-2.0,-2.0,-2.0,...,hairacademy2.com/financialaid,2,-2.0,hairacademy2.com,MD,24,11352.0,0,0.0,20748
6,6,436067,36.309563,-95.322932,National Accrediting Commission of Cosmetology...,None,1,-2.0,-2.0,-2.0,...,www.pryorbeautycollege.com/npcalc.htm,6,-2.0,www.pryorbeautycollege.com/,OK,40,3876.0,0,0.0,74361
7,7,436191,38.521026,-90.418840,Accrediting Commission of Career Schools and C...,None,11,14.0,6.0,11.0,...,www.vatterott.edu/calculator/,4,-2.0,www.vatterott.edu,MO,29,9628.0,0,0.0,63127
8,8,436270,40.335502,-111.712373,Accrediting Council for Continuing Education &...,None,9,-2.0,-2.0,-2.0,...,www.steinered.com/utah/npcalc_west.htm,7,-2.0,www.steinered.com,UT,49,6968.0,0,0.0,84057
9,9,164915,42.339481,-71.133025,New England Association of Schools and College...,None,1,26.0,18.0,0.0,...,None,1,-2.0,www.bgsp.edu/,MA,25,14175.0,0,0.0,2446


In [275]:
student_df = pd.read_csv('./data/school.csv')

In [276]:
student_df.head(10)

,id,location.lat,location.lon,accreditor,alias,branches,carnegie_basic,carnegie_size_setting,carnegie_undergrad,city,...,price_calculator_url,region_id,religious_affiliation,url,state,state_fips,tuition_revenue_per_fte,under_investigation,women_only,zip
0,121983,37.781238,-122.267523,Western Association of Schools and Colleges Se...,NaN,28,26.0,6.0,5.0,Alameda,...,tcc.noellevitz.com/edmc/Argosy University Net ...,8,-2.0,www.argosy.edu/sanfrancisco,CA,6,12380.0,0,0.0,94501
1,120838,34.060700,-118.299841,Accrediting Council for Independent Colleges a...,NaN,1,29.0,6.0,11.0,Los Angeles,...,www.psuca.edu/node/100,8,-2.0,www.psuca.edu,CA,6,13784.0,0,0.0,90010
2,434973,39.189991,-76.815804,North Central Association of Colleges and Scho...,NaN,38,29.0,6.0,11.0,Columbia,...,www.phoenix.edu/tuition_and_financial_options/...,2,-2.0,www.phoenix.edu,MD,24,10131.0,0,0.0,21045
3,436012,40.708670,-73.627681,NaN,NaN,2,-2.0,-2.0,-2.0,Hempstead,...,www.franklincareer.edu,2,-2.0,www.franklincareer.edu,NY,36,6935.0,0,0.0,11550
4,436021,42.986115,-87.870663,Montessori Accreditation Council for Teacher E...,NaN,1,-2.0,-2.0,-2.0,Milwaukee,...,www.montessori6-12ami.org,3,-2.0,www.montessori6-12ami.org,WI,55,6063.0,0,0.0,53207
5,436030,38.839695,-76.948468,National Accrediting Commission of Cosmetology...,NaN,1,-2.0,-2.0,-2.0,Hillcrest Heights,...,hairacademy2.com/financialaid,2,-2.0,hairacademy2.com,MD,24,11352.0,0,0.0,20748
6,436067,36.309563,-95.322932,National Accrediting Commission of Cosmetology...,NaN,1,-2.0,-2.0,-2.0,Pryor,...,www.pryorbeautycollege.com/npcalc.htm,6,-2.0,www.pryorbeautycollege.com/,OK,40,3876.0,0,0.0,74361
7,436191,38.521026,-90.418840,Accrediting Commission of Career Schools and C...,NaN,11,14.0,6.0,11.0,Sunset Hills,...,www.vatterott.edu/calculator/,4,-2.0,www.vatterott.edu,MO,29,9628.0,0,0.0,63127
8,436270,40.335502,-111.712373,Accrediting Council for Continuing Education &...,NaN,9,-2.0,-2.0,-2.0,Orem,...,www.steinered.com/utah/npcalc_west.htm,7,-2.0,www.steinered.com,UT,49,6968.0,0,0.0,84057
9,164915,42.339481,-71.133025,New England Association of Schools and College...,NaN,1,26.0,18.0,0.0,Brookline,...,NaN,1,-2.0,www.bgsp.edu/,MA,25,14175.0,0,0.0,2446


#### Add primary keys to database

In [283]:
primary_key_add = """ALTER TABLE
                        student
                    ADD PRIMARY KEY
                        (id);"""

In [284]:
conn.execute(primary_key_add)

In [290]:
primary_key_add_zip = """ALTER TABLE
                        school
                    ADD PRIMARY KEY
                        (id,zip);"""

In [291]:
conn.execute(primary_key_add_zip)

In [293]:
primary_key_add_aid = """ALTER TABLE
                        aid
                    ADD PRIMARY KEY
                        (id);"""

In [294]:
conn.execute(primary_key_add_aid, conn)

In [296]:
primary_key_add_rpmt = """ALTER TABLE
                        repayment
                    ADD PRIMARY KEY
                        (id);"""

In [297]:
conn.execute(primary_key_add_rpmt, conn)